In [1]:
import numpy as np # linear algebra
from tqdm.notebook import tqdm
import re, os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader,SequentialSampler
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import DebertaTokenizer, DebertaModel, AdamW, get_linear_schedule_with_warmup, DebertaForSequenceClassification, DebertaConfig

from datasets import Dataset

In [4]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# Text cleaning function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text)  # Remove extra whitespace
    text = " ".join([stemmer.stem(word) for word in text.split() if word not in stop_words])  # Remove stopwords and stem
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
model_name = "microsoft/deberta-base"
tokenizer = DebertaTokenizer.from_pretrained(model_name,truncation=True, do_lower_case=True)

In [5]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [3]:
batch_size = 32
threshold = 0.10

print("Starting preprocessing...")
check_df = pd.read_csv("test_unlabeled.csv")

check_df["Cleaned Text"] = check_df["Title"] + " " + check_df["Abstract"]

# check_df["text"] = check_df["Title"] + " " + check_df["Abstract"]
# check_df["Cleaned Text"] = check_df["text"].apply(clean_text)

check_text = check_df["Cleaned Text"].values

check_token_ids = []
check_attention_masks = []
for description in check_text:
  encoding_dict = tokenizer.encode_plus(
     description,
      add_special_tokens=True,
      max_length=256,
      padding='max_length',  
      truncation=True,       
      return_attention_mask=True,
      return_tensors="pt"
  )
  check_token_ids.append(encoding_dict["input_ids"]) 
  check_attention_masks.append(encoding_dict["attention_mask"])

check_token_ids = torch.cat(check_token_ids, dim=0)
check_attention_masks = torch.cat(check_attention_masks, dim=0)

check_set = TensorDataset(check_token_ids, check_attention_masks)
check_dataloader = DataLoader(check_set, sampler=SequentialSampler(check_set), batch_size=batch_size)

print("Finished preprocessing, now loading model...")

# model = BertForSequenceClassification.from_pretrained(
#     model_name, 
#     num_labels=2, 
#     state_dict=torch.load(f"./augbertfiles/finetuned_augBERT_epoch_5.model", map_location="cuda")
# )
model = DebertaForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2
)
model.load_state_dict(torch.load(f"./augdebertfiles/finetuned_deBERTa_epoch_1.model", map_location="cuda"))

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Set the model to evaluation mode
model.eval()

print("Model loaded, now predicting...")

predictions = []

# No gradient calculation is needed during inference
with torch.no_grad():
    progress = tqdm(check_dataloader, desc="Predicting")
    for batch in progress:
        batch = tuple(b.to("cuda") for b in batch)

        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}

        # Get model outputs
        outputs = model(**inputs)
        logits = outputs.logits  # Extract logits

        probs = F.softmax(logits, dim=1)
        # Convert logits to predicted class
        preds = torch.argmax(probs, dim=1).cpu().numpy()

        # CUSTOM CLASS THRESHOLDING
        # class_1_probs = probs[:, 1].cpu().numpy()  
        # preds = (class_1_probs >= threshold).astype(int)
        
        predictions.extend(preds)

# Convert lists to NumPy arrays
predictions = np.array(predictions)

# predictions = np.concatenate(predictions, axis=0)
# predictions = np.argmax(predictions, axis=1).flatten()

check_df["Label"] = predictions
check_df = check_df[["PMID", "Label"]]

check_df.to_csv("submission-aug.csv", index=False)


Starting preprocessing...
Finished preprocessing, now loading model...


c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for DebertaForSequenceClassification:
	Missing key(s) in state_dict: "deberta.embeddings.word_embeddings.weight", "deberta.embeddings.LayerNorm.weight", "deberta.embeddings.LayerNorm.bias", "deberta.encoder.layer.0.attention.self.q_bias", "deberta.encoder.layer.0.attention.self.v_bias", "deberta.encoder.layer.0.attention.self.in_proj.weight", "deberta.encoder.layer.0.attention.self.pos_proj.weight", "deberta.encoder.layer.0.attention.self.pos_q_proj.weight", "deberta.encoder.layer.0.attention.self.pos_q_proj.bias", "deberta.encoder.layer.0.attention.output.dense.weight", "deberta.encoder.layer.0.attention.output.dense.bias", "deberta.encoder.layer.0.attention.output.LayerNorm.weight", "deberta.encoder.layer.0.attention.output.LayerNorm.bias", "deberta.encoder.layer.0.intermediate.dense.weight", "deberta.encoder.layer.0.intermediate.dense.bias", "deberta.encoder.layer.0.output.dense.weight", "deberta.encoder.layer.0.output.dense.bias", "deber

In [14]:
preds = pd.read_csv("submission-aug.csv")

unique, counts = np.unique(preds['Label'], return_counts=True)
print(dict(zip(unique, counts)))

{0: 1097}
